# Functional Data Structures

A **Functional data structure** is operated on using only **pure functions**.
Therefore, functional data structures are by definition **immutable**.

Example:

The empty list (List() or Nil) is as eternal and immutable as the integer values 3 or 4.


## singly linked list

In [1]:

sealed trait List[+A] //형식 A에 대해 매개변수화된 List자료형식
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {
    def sum(ints: List[Int]): Int = ints match {
        case Nil => 0
        case Cons(x,xs) => x + sum(xs)
    }
    
    def product(ds: List[Double]): Double = ds match {
        case Nil => 1.0
        case Cons(0.0, _) => 0.0
        case Cons(x,xs) => x * product(xs)
    }
    
    def apply[A](as: A*): List[A] = {
        if(as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    }
//     def empty[A](): List[A] =
//         Nil
    
//     def isEmpty[A](l: List[A]): Boolean = l match {
//         case Nil => true
//         case _ => false
//     }
}

defined trait List
defined object Nil
defined class Cons
defined object List

In [2]:
sealed trait List[+A] //sealed trait 인터페이스처럼 타입정의?
                      //sealed 하나의 파일안에 모든 차일드(상속받아 생성되는)가 정의되고 더 상속받을 수 없음

final case object Nil extends List[Nothing]
final case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {   //companion object 
    def apply[A](as: A*): List[A] = {
        if(as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    }
}

defined trait List
defined object Nil
defined class Cons
defined object List

In [3]:
val ex1: List[Double] = Nil
val ex2: List[Int] = Cons(1, Nil)
val ex3: List[String] = Cons("a", Cons("b", Nil))

ex1: List[Double] = Nil
ex2: List[Int] = Cons(1,Nil)
ex3: List[String] = Cons(a,Cons(b,Nil))

### Variance Annotation

In trait List[+A],
              ^
              variance  annotation indicating that A is a covariant.
              
Covariant: For all types X and Y,
 - if X <: Y, hen List[X] <: List[Y].
 
If not annotated, parameter is *invariant*, meaning there is no subtyping relationship List[X] and List[Y].

Nil extends List[Nothing]. Since Nothing is a subtype of all types, in conjunction with the variance annotation,
 - Nil can be considered a sub type of any List[XXX]

### Companion Object
We'll often declare a companion object in addition to our data type and its data constructors.

The companion object is the one with *the same name* as the data type(in this case List) where we put various convenience functions for creating or working with values of the data type.

```scala
object List {
    def sum(xs: List[Int]): Int = ???
    def product(xs: List[Double]): Double = ???
    def apply[A](as: A*): List[A] = ???
    ...
}
```

### Pattern Matching

```scala
def sum(xs: List[Int]): Int = xs match {
    case Nil => 0
    case Cons(x, xsl) => x + sum(xsl)
}
```

Pattern mathcing descends into the structure of the expression it examinges and extract subexpressions of that structure.
```scala
target match { pattern => result; ...}
```
If the target matches the pattern in a case, the result of that case becomes the result of the entire match expression.

If multiple patterns match the target, Scala chooses the first matching case.

In [5]:
// Pattern Matching Example
List(1,2,3) match { case Cons(h, _) => h}

// Variable pattern, _, matches any expression.
List(1,2,3) match { case _ => 42}

// Data constructor pattern in conjunction with variables to capture 
// or bind a subexpression of the target
List(1,2,3) match { case Cons(_, t) => t}

// List(1,2,3) match { case Nil => 42} 
// scala.MatchError

res4_0: Int = 1
res4_1: Int = 42
res4_2: List[Int] = Cons(2,Cons(3,Nil))

// Exercise
what will be 

### Variadic functions

The function apply in the comanion object List is a variadic function, meaning it accepts zero or more arguments of type A:

---
```scala
def apply[A](as: A*): List[A] = 
    if (as.isEmpty) Nil
    else Cons(as.head, apply(as.tail: _*)) // _ 뒤에 *는 시퀀스 타입인 _을 분해해주는 것을 뜻함 
```
---

Variadic functions provides alittle *syntatic sugar* for creating and passing a `Seq` of elemnts explicitly.

## Data Sharing and Persistent Data Structures

```scala
def tail[A](as: List[A]): List[A] = as match {
    case Cons(h, t) => t
    case Nil => throw new UnsupportedOperationException;
}
```

### Efficiency of Data Sharing

**Vector** in Scla standard library is a purely functional sequence implementation
벡터는 각종 오퍼레이션의 시간복잡도가 상수임


append adds all the elements of one list to the end of another:
```scala
def append[A](a1: List[A], a2: List[A]): List[A] = a1 match {
    case Nil => a2
    case Cons(h, t) => Cons(h, append(t, a2))
}
```

In [6]:
def append[A](a1: List[A], a2: List[A]): List[A] = a1 match {
    case Nil => a2
    case Cons(h, t) => Cons(h, append(t, a2))
}

defined function append

In [7]:
append(List(1,2,3), List(4,5))

res6: List[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))

Exercise

Implement a function, *init*, htat returns a List consisting of all but the last element of a List.
```scala
init(list(1,2,3,4)) == List(1,2,3)
```

In [8]:
def init[A](as: List[A]): List[A] = as match {
    case Cons(_, Nil) => Nil
    case Cons(h, t) => Cons(h, init(t))
    case Nil => throw new UnsupportedOperationException;
}

defined function init

In [9]:
def dropWhile_[A](l: List[A], f: A => Boolean): List[A] = l match {
    case Cons(h, t) if (f(h)) => dropWhile_(t, f)
    case e @ Cons(h, t) => e
    case _ => Nil
}

defined function dropWhile_

Improving Type inference for HOFs

Use curried form to maximize type inference.

In [10]:
def dropWhile[A](l: List[A])(f: A => Boolean): List[A] = l match {
    case Cons(h, t) if (f(h)) => dropWhile(t)(f)
    case e @ Cons(h, t) => e
    case _ => Nil
}

defined function dropWhile

Don't Repeat Yourself (DRY)

``` scala
def sum(ints: List[Int]): Int = ints match {
    case Nil => 0
    case Cons(x,xs) => x + sum(xs)
}
    
def product(ds: List[Double]): Double = ds match {
    case Nil => 1.0
    case Cons(x,xs) => x * product(xs)
}
```

### foldRight
stack overflow의 위험성이 있음

In [11]:
def foldRight[A,B](as: List[A], z: B)(f: (A, B) => B): B = as match {
    case Nil => z
    case Cons(h,t) => f(h, foldRight(t,z)(f))
}

defined function foldRight

In [12]:
def sum(ns: List[Int]) = 
    foldRight(ns, 0)((x,y) => x+y)
def product(ns: List[Double]) = 
    foldRight(ns, 1.0)(_ * _)

defined function sum
defined function product

### foldLeft

In [13]:
def foldLeft[A,B](as: List[A], z: B)(f: (B, A) => B): B = as match {
    case Cons(h,t) => foldLeft(t, f(z, h))(f)
    case Nil => z
}

defined function foldLeft

### Exercise

In [14]:
def length[A](as: List[A]): Int = 
    foldRight(as, 0)((_, acc) => 1 + acc)

defined function length

In [15]:
// 요거는 안뒤집힘 그대로임 foldRight로 넘겨준 f가 맨끝원소부터 적용되기 때문에 그대로 쌓임
def reverse[A](as: List[A]): List[A] =
    foldRight(as, Nil: List[A])((a, acc) => Cons(a, acc))
// or foldRight(as, List[A]())((a, acc) => Cons(a, acc))
// or foldRight(as, List.empty[A])((a, acc) => Cons(a, acc))

defined function reverse

In [16]:
def reverse[A](as: List[A]): List[A] =
    foldLeft(as, Nil: List[A])((acc, a) => Cons(a, acc))
// or foldLeft(as, List[A]())((acc, a) => Cons(a, acc))
// or foldLeft(as, List.empty[A])((acc, a) => Cons(a, acc))

defined function reverse

# 04-19

In [17]:

def flatten[A](ffa: List[List[A]]): List[A] = ffa match{
    case Cons(h,t)=> append(h, flatten(t))
    case Nil => Nil
}

def join[A](ffa: List[List[A]]): List[A] = {
    foldRight(ffa, List.empty[A])((a, acc) => append(a, acc))
}

def join_[A](ffa: List[List[A]]): List[A] = 
    foldRight(ffa, List.empty[A])((a, acc) => append(a, acc))

defined function flatten
defined function join
defined function join_

## more exercise

In [18]:
def incOne(as: List[Int]): List[Int] = as match {
    case Cons(h, t) => Cons(h+1, incOne(t))
    case Nil => Nil
}

defined function incOne

In [19]:
incOne(List(1,2,3,4))

res18: List[Int] = Cons(2,Cons(3,Cons(4,Cons(5,Nil))))

In [20]:
def doubleToString(as: List[Double]): List[String] = as match {
    case Cons(h, t) => Cons(h.toString, doubleToString(t))
    case Nil => Nil
}

defined function doubleToString

## map

In [21]:
def map[A, B](fa: List[A])(f: A => B): List[B] = fa match {
    case Cons(h, t) => Cons(f(h), map(t)(f))
    case Nil => Nil
}

defined function map

In [22]:
map(List(1,2,3,4,5))(_ + 1)

res21: List[Int] = Cons(2,Cons(3,Cons(4,Cons(5,Cons(6,Nil)))))

In [23]:
map(List(1,2,3,4,5))(_.toString)

res22: List[String] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))

### map fold version

In [24]:
def map[A,B](as: List[A])(f: A => B): List[B] = 
    foldRight(as, List.empty[B])((a, acc) => Cons(f(a), acc))

defined function map

## flat map
```scala
def flatMap[A,B](as: List[A])(f: A => List[B]): List[B]
```

In [25]:
def flatMap[A,B](as: List[A])(f: A => List[B]): List[B] = as match {
    case Cons(h, t) => append(f(h), flatMap(t)(f))
    case Nil => Nil
}
// def flatten[A]: List[A] = List.flatten(as)
// def flatMap[A,B](as: List[A])(f: A => List[B]): List[B] = {
//     flatten(map(as)(f))
// }

defined function flatMap

---
map - join - pure
flatmap - pure

---

## filter

In [31]:
def pure[A](a: A): List[A] = List(a)
def filter[A](as: List[A])(f: A => Boolean): List[A] = {
    flatMap(as)(a => if(f(a)) pure(a) else Nil) // apply가 정의되어 있으면 Cons 대신 List(a)해도됨
}

defined function pure
defined function filter

In [32]:
filter(List(1,2,3,4,5))(_ % 2 ==0)

res31: List[Int] = Cons(2,Cons(4,Nil))

## more exercises

In [33]:
def zipWithInt(as: List[Int], bs: List[Int]): List[Int] = (as, bs) match {
    case (Cons(h1, t1), Cons(h2, t2)) => Cons(h1+h2, zipWithInt(t1, t2))
    case _ => Nil
}

defined function zipWithInt

## 숙제
- zipWith 구하기
- tails 구하기
- subset 구하기
- permutation 구하기

In [35]:
def zipWith[A, B, C](as: List[A], bs: List[B])(f: (A, B) => C): List[C] = (as, bs) match {
    case (Cons(h1, t1), Cons(h2, t2)) => Cons(f(h1,h2), zipWith(t1, t2)(f))
    case _ => Nil
}

defined function zipWith

In [36]:
// {1,2,3} -> {{1,2,3}, {2,3}, {3}}
def tails[A](as: List[A]): List[List[A]] = ???

defined function tails

# Algebraic Data Types (ADTs)
**List** is just one example of what's called an **algebraic data type(ADT)**.

**Sum type (OR type)**:
- a data type defined by the sum or union of its data constructors
- enum, Option, Either, List, Tree, etc.

**Product type (AND type)**:
- a data type defined by the product of other types
- tuple, a (case) class with arguments

## The Algebra of Data Types
- 0 - void
- 1 - unit (())
- 2 - boolean

|algebra|ADT|
|--------------:|:---------------|
|(a * b) * c = a * (b * c) | ((a, b), c) = (a, (b, c))|
|a * 1 = a (right unit) | (a, ()) = a|
|a + 0 = a | Either(a, void)|
|a * (b + c) = a * b + a * c | (a, Either[b, c]) = Either[(a, b), (a, c)]|
|a^(b^c) = a^(bc) | c => b => a = (b, c) => a|

```
l(a) = 1 + a * l(a)
l(a) - a * l(a) = 1
l(a)(1 - a) = 1
l(a) = 1 / (1 - a) // geometric sequence

data List a = Nil | Cons a (List a)
```

## 더 숙제 
binary search tree 구현
leaf엔 값이 없고 branch에만 값이 있음, leaf의 left, right, value는 모두 empty(not exception)
- insert
- delete
- find